# 自作関数・自作層・自作損失関数
様々な深層学習モデルを実装するにあたり，ライブラリに未だ用意されていない演算を利用したい場合や，既存関数を改良して利用したいという状況は多く訪れます．このノートブックでは，そのような場合に自前でカスタム実装を行う方法を概観します．

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## 自作関数
PyTorchの演算は自動微分機能を備えているため，基本的には通常のPython関数と同様に任意の演算を関数化することができます．

ここでは例として[Mish関数](https://arxiv.org/ftp/arxiv/papers/1908/1908.08681.pdf)を実装してみます．Mish関数は近年ReLUやSwishを種々のタスクにおいて凌駕する性能を達成したとして発表された活性化関数です．


$Mish(x)=x \cdot tanh(softplus(x)) = x \cdot tanh(ln(1+e^x)).$

In [2]:
def mish(x):
  return x * (F.softplus(x)).tanh()

In [3]:
x = torch.randn(5)
print(x)
y = mish(x)
print(y)

tensor([ 0.0788, -0.1664, -0.4991,  0.5902, -0.3639])
tensor([ 0.0493, -0.0909, -0.2205,  0.4571, -0.1760])


## 自作層
パラメータを保有した層として関数を実装したい場合，以下のようにして所望の層を作成することができます．


1.   初期化時に必要なパラメータを定義しておく．
2.   forward関数を実装する．

同じく自動微分機能の恩恵により，逆伝播を顕に記述する必要はありません．
torch.nn.Parameterでラップしたtorch.Tensorを用いてパラメータを定義することで，誤差逆伝播の勾配グラフに自動でそれらのパラメータが追加されます．したがって，ユーザは特に意識することなく学習可能なパラメータを設計することができます．

例として，先ほどのMish関数が自身でその内部のスケールを学習できるように調整してみましょう．


$ParametricMish(x)=x \cdot tanh(softplus(\alpha x)) = x \cdot tanh(ln(1+e^{\alpha x})).$

In [4]:
class ParametericMish(nn.Module):
  def __init__(self, feature_dims):
    super().__init__()
    self.alpha = nn.Parameter(torch.ones(feature_dims))
  
  def forward(self, x):
    return x * (F.softplus(self.alpha * x)).tanh()


In [5]:
d = 5
pmish = ParametericMish(5)
x = torch.randn(5)
print(x)
y = pmish(x)
print(y)

tensor([-1.6502, -0.9197,  1.1170, -0.6380,  0.6981])
tensor([-0.2869, -0.2975,  0.9890, -0.2555,  0.5593], grad_fn=<MulBackward0>)


In [6]:
# 逆伝播してみる
y.sum().backward()
print(pmish.alpha.grad) # 勾配が計算されていることを確認

tensor([0.4254, 0.2159, 0.2032, 0.1182, 0.1165])


## 自作損失関数
同様の要領で，損失関数を自作することも可能です．
損失関数というとかしこまった印象を受けるかもしれませんが，実際は任意の入力を受け取って損失となるスカラー値を返す関数であり，これまでの内容で問題なく実装できます．
基本的に損失関数は評価指標ですので，内部にパラメータを保持することはありません．一方，使い勝手の観点から関数よりもクラスとして定義されることが多いため，ここでは例としてJSDivergenceを実装してみます．

JSDivergenceは確率分布間のある種の距離指標であり，KLDivergenceの非対称性を解消するために考案されたものです．

入力となる確率分布を$P(x)$, $Q(x)$，2つの確率分布間のKLDivergenceを$D_{KL}(P\|Q)$としたとき，JSDivergenceは以下のように表されます．

$M(x)=\frac{P(x)+Q(x)}{2},$

$D_{JS}(P\|Q) = \frac{1}{2}\left(D_{KL}(P\|M)+D_{KL}(Q\|M)\right).$

In [7]:
class JSDivergence(nn.Module):
  def __init__(self):
    super().__init__()
  
  def forward(self, p, q):
    r"""
    バッチごとにJSDIvergenceを計算し，その平均を全体の損失として返す
    p, q: (B, K)
    """
    r = (p + q) / 2
    return ((self.kl_divergence(p, r) + self.kl_divergence(q, r)) / 2).mean()
  
  def kl_divergence(self, p, q):
    r"""
    KLDivergenceの計算
    """
    return (p * ((p / q.clamp(min=1e-10)).clamp(min=1e-10)).log()).sum(dim=-1) # nan防止


In [8]:
# B=3, outcome=5 の離散確率分布をランダムに2つ生成
p = torch.randn(3, 5)
p = F.softmax(p, dim=-1)
p.requires_grad = True
print("[p]")
print(p)
print(p.data.sum(dim=-1)) # 各バッチにおいて確率の和が1になることを確認
print()

q = torch.randn(3, 5)
q = F.softmax(q, dim=-1)
q.requires_grad = True
print("[q]")
print(q)
print(q.data.sum(dim=-1)) # 各バッチにおいて確率の和が1になることを確認

[p]
tensor([[0.0377, 0.1408, 0.5400, 0.1241, 0.1574],
        [0.1323, 0.4633, 0.1190, 0.0702, 0.2152],
        [0.2938, 0.0513, 0.3680, 0.1343, 0.1526]], requires_grad=True)
tensor([1.0000, 1.0000, 1.0000])

[q]
tensor([[0.2081, 0.2939, 0.2150, 0.0429, 0.2401],
        [0.1312, 0.3823, 0.2424, 0.1049, 0.1392],
        [0.6675, 0.1413, 0.0751, 0.0824, 0.0337]], requires_grad=True)
tensor([1.0000, 1.0000, 1.0000])


In [9]:
js_div = JSDivergence()
# 損失の計算
loss = js_div(p, q)
print(loss)

tensor(0.0801, grad_fn=<MeanBackward0>)


In [10]:
# 誤差逆伝播
loss.backward()
print(p.grad)
print(q.grad)

tensor([[-0.1970, -0.0724,  0.0597,  0.0661, -0.0389],
        [ 0.0007,  0.0152, -0.0696, -0.0368,  0.0324],
        [-0.0820, -0.1051,  0.0846,  0.0358,  0.0822]])
tensor([[ 0.0878,  0.0503, -0.0938, -0.1111,  0.0315],
        [-0.0007, -0.0168,  0.0490,  0.0301, -0.0402],
        [ 0.0547,  0.0639, -0.1804, -0.0456, -0.1693]])
